# Prompt engineering

Prompts consist of three main components: 

* __Instructions__ that describe the task requirements, goals, and format of input/output. They explain the task to the model unambiguously. 
* __Examples__ that demonstrate the desired input-output pairs. They provide diverse demonstrations of how different inputs should map to outputs. 
* __Input__ that the model must act on to generate the output.


## Zero-shot Prompting

* No examples provided; rely on the models training
* Leverages the models pre-training
* Works for simple tasks, but struggles with complex reasoning 

In [1]:
from langchain.globals import set_llm_cache
from langchain_community.chat_models import ChatOllama
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from IPython.display import display, Markdown
from langchain import PromptTemplate

In [2]:
llm = ChatOllama(base_url="http://192.168.178.84:11434", model="gemma:7b", temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [4]:
prompt = PromptTemplate(input_variables=["text"], template="Classify the sentiment of this text: {text}")
chain = prompt | llm

In [5]:
result = chain.invoke({"text": "I hate the movie Batman & Robin, it was terrible!"})

In [7]:
display(Markdown(result.content))

The sentiment of the text is negative. The speaker is expressing hate and negativity towards the movie Batman & Robin.

## Few-shot Prompting

* Provide a few demos of input and desired output
* Shows desired reasoning format
* Tripled accuracy on grade-school math

In [8]:
prompt_1 = """
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
 
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:
"""

In [9]:
result_1 = llm.invoke(prompt_1)
display(Markdown(result_1.content))

Sure, here is the revised text:

A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word "whatpu" is:

We were traveling in Africa and we saw these very cute whatpus.

To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word "farduddle" is:

We were hiking in the forest and we saw a group of people farddling down the hill.

## Chain-of-Thought (CoT)

In [3]:
prompt_2 = """
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
"""

In [5]:
result_2 = llm.invoke(prompt_2)
display(Markdown(result_2.content))

Sure, here is the solution:

1. You bought 10 apples, gave 2 apples to the neighbor and 2 apples to the repairman, so you have 10 - 2 - 2 = 6 apples left.
2. You bought 5 more apples, so you have 6 + 5 = 11 apples.
3. You ate 1 apple, so you have 11 - 1 = 10 apples left.

Therefore, you have a total of 10 apples remaining.

In [6]:
prompt_3 = """
Given this context: ```Jeff and Tommy are neighbors. Tommy and Eddy are not neighbors.``` and the following query: ```Are Jeff and Eddy neighbors?```. 
Please answer the questions in the query and explain your reasoning.
If there is not enough informaton to answer, please say
"I do not have enough information to answer this questions."
"""

In [7]:
result_3 = llm.invoke(prompt_3)
display(Markdown(result_3.content))

**Answer:** I do not have enough information to answer this question.

The text provided does not mention any relationship between Jeff and Eddy, therefore I cannot answer the query.

## Least-to-most Prompting

## Self-consistency

## Chain-of-density

## Chain-of-verification (CoV)

## Active Prompting

## Tree-of_Thought

## Verifiers

## Fine-Tuning